In [1]:
import pandas as pd
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [2]:
df_ = pd.read_excel('gezinomi.xlsx')
df = df_.copy() 

In [3]:
def general_information(df):
    print("_____ INFO _____")
    print(df.info(), end='\n\n')
    print("_____ DESCRIPTION _____")
    print(df.describe().T, end='\n\n')
    print("_____ SHAPE _____")
    print(df.shape,end ='\n\n')
    print("_____ MISSING _____")
    print(df.isnull().sum().sort_values(ascending=False), end='\n\n')
    print("_____ QUANTILE _____")
    print(df.quantile([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99, 1]).T, end='\n\n')
    print("_____ UNIQUE _____")
    print(df.nunique(), end='\n\n')
    print("_____ HEAD _____")
    print(df.head(), end='\n\n')
    print("_____ TAIL _____")
    print(df.tail(), end='\n\n')
    print("_____ SAMPLE _____")
    print(df.sample(5), end='\n\n')

general_information(df)

_____ INFO _____
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59164 entries, 0 to 59163
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   SaleId              59164 non-null  int64         
 1   SaleDate            59164 non-null  datetime64[ns]
 2   CheckInDate         59164 non-null  datetime64[ns]
 3   Price               59151 non-null  float64       
 4   ConceptName         59164 non-null  object        
 5   SaleCityName        59164 non-null  object        
 6   CInDay              59164 non-null  object        
 7   SaleCheckInDayDiff  59164 non-null  int64         
 8   Seasons             59164 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 4.1+ MB
None

_____ DESCRIPTION _____
                      count           mean           std      min            25%            50%            75%           max
SaleId              591

In [4]:
df.SaleCityName.unique()

array(['Antalya', 'İzmir', 'Diğer', 'Aydın', 'Muğla', 'Girne'],
      dtype=object)

In [5]:
df.SaleCityName.value_counts()

Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: SaleCityName, dtype: int64

In [6]:
df.ConceptName.unique()

array(['Herşey Dahil', 'Yarım Pansiyon', 'Oda + Kahvaltı'], dtype=object)

In [7]:
df.ConceptName.value_counts()

Herşey Dahil      53186
Yarım Pansiyon     3559
Oda + Kahvaltı     2419
Name: ConceptName, dtype: int64

In [8]:
df.groupby('SaleCityName')['Price'].sum()

SaleCityName
Antalya    2.041911e+06
Aydın      5.732960e+05
Diğer      1.545723e+05
Girne      2.706503e+04
Muğla      6.658422e+05
İzmir      1.659348e+05
Name: Price, dtype: float64

In [9]:
df.groupby('ConceptName')['Price'].count()

ConceptName
Herşey Dahil      53180
Oda + Kahvaltı     2414
Yarım Pansiyon     3557
Name: Price, dtype: int64

In [10]:
df.groupby('SaleCityName')['Price'].mean()

SaleCityName
Antalya    64.521474
Aydın      53.855896
Diğer      47.707497
Girne      59.483579
Muğla      62.461746
İzmir      66.267905
Name: Price, dtype: float64

In [11]:
df.groupby('ConceptName')['Price'].mean()

ConceptName
Herşey Dahil      62.672260
Oda + Kahvaltı    50.252010
Yarım Pansiyon    49.030742
Name: Price, dtype: float64

In [12]:
df.groupby(['SaleCityName', 'ConceptName'])['Price'].mean()

SaleCityName  ConceptName   
Antalya       Herşey Dahil      64.519028
              Oda + Kahvaltı    63.504816
              Yarım Pansiyon    67.190613
Aydın         Herşey Dahil      53.995467
              Oda + Kahvaltı    34.458787
              Yarım Pansiyon    30.016543
Diğer         Herşey Dahil      84.771093
              Oda + Kahvaltı    37.599082
              Yarım Pansiyon    42.113440
Girne         Herşey Dahil      97.680970
              Oda + Kahvaltı    39.775536
              Yarım Pansiyon    53.247838
Muğla         Herşey Dahil      63.020053
              Oda + Kahvaltı    59.037763
              Yarım Pansiyon    45.120924
İzmir         Herşey Dahil      74.701519
              Oda + Kahvaltı    41.320342
              Yarım Pansiyon    59.610053
Name: Price, dtype: float64

In [13]:
df["EB_Score"] = pd.cut(df.SaleCheckInDayDiff, bins=[-1, 7, 30, 90, df.SaleCheckInDayDiff.max()], labels = ["Last Minuters", "Potential Planners", "Planners", "Early bookers"])

In [14]:
new_df = df.groupby(['SaleCityName', 'ConceptName', 'EB_Score']).agg({'Price': ['mean', 'count']})
new_df

Price       
                                                     mean  count
SaleCityName ConceptName    EB_Score                            
Antalya      Herşey Dahil   Last Minuters       62.752916  14148
                            Potential Planners  64.903700   8874
                            Planners            67.879960   4490
                            Early bookers       66.494885   3281
             Oda + Kahvaltı Last Minuters       65.353740    503
...                                                   ...    ...
İzmir        Oda + Kahvaltı Early bookers       66.928111      3
             Yarım Pansiyon Last Minuters       57.788126    636
                            Potential Planners  64.804043    198
                            Planners            60.507887     47
                            Early bookers       65.905583     14

[72 rows x 2 columns]

In [15]:
agg_df = df.groupby(['SaleCityName', 'ConceptName', 'Seasons']).agg({'Price': ['mean']}).sort_values(by=[('Price', 'mean')], ascending=False)
agg_df.head()

Price
                                           mean
SaleCityName ConceptName    Seasons            
Girne        Herşey Dahil   High     103.935448
                            Low       90.935944
İzmir        Yarım Pansiyon High      87.657303
Diğer        Herşey Dahil   Low       87.310882
                            High      83.787273

In [16]:
agg_df.index
agg_df.reset_index(inplace = True)

In [17]:
agg_df["sales_level_based"] = (agg_df.SaleCityName + "_" + agg_df.ConceptName + "_" + agg_df.Seasons).apply(lambda x: x.upper())
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price,sales_level_based
,,,,mean,
0,Girne,Herşey Dahil,High,103.935448,GIRNE_HERŞEY DAHIL_HIGH
1,Girne,Herşey Dahil,Low,90.935944,GIRNE_HERŞEY DAHIL_LOW
2,İzmir,Yarım Pansiyon,High,87.657303,İZMIR_YARIM PANSIYON_HIGH
3,Diğer,Herşey Dahil,Low,87.310882,DIĞER_HERŞEY DAHIL_LOW
4,Diğer,Herşey Dahil,High,83.787273,DIĞER_HERŞEY DAHIL_HIGH


In [18]:
agg_df.columns = agg_df.columns.droplevel(1)

In [19]:
agg_df["SEGMENT"] = pd.qcut(agg_df["Price"], 4, labels=["D", "C", "B", "A"])
agg_df.groupby("SEGMENT").agg({"Price": ["mean", "max", "sum"]})

Price                        
              mean         max         sum
SEGMENT                                   
D        33.366288   39.479630  300.296592
C        44.887654   54.141797  403.988887
B        60.274498   64.920065  542.470478
A        82.467833  103.935448  742.210498

In [20]:
customer1 = "ANTALYA_HERŞEY DAHIL_HIGH"
agg_df[agg_df.sales_level_based == customer1]

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
9,Antalya,Herşey Dahil,High,64.920065,ANTALYA_HERŞEY DAHIL_HIGH,B


In [21]:
customer2 = "GIRNE_YARIM PANSIYON_LOW"
agg_df[agg_df.sales_level_based == customer2]

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
19,Girne,Yarım Pansiyon,Low,48.578903,GIRNE_YARIM PANSIYON_LOW,C
